In [50]:
# Langkah Pertama : Load Data (Memuat Data yang sudah ada )

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import GRU, Dense
import datetime
import os
import tensorflow as tf

# Disable eager execution
tf.compat.v1.disable_eager_execution()

# Load the data
file_path = 'Tether_to_IDR_Conversion.xlsx'
data = pd.read_excel(file_path)

# Convert date to datetime format
data['Tanggal'] = pd.to_datetime(data['Tanggal'], format='%d/%m/%Y')
data.set_index('Tanggal', inplace=True)

# Ensure the data is sorted by date
data = data.sort_index()

# Display the first few rows of the data
display(data.head())


,Terakhir_IDR,Pembukaan_IDR,Tertinggi_IDR,Terendah_IDR
Tanggal,,,,
2024-04-24,16084.993428,16093.039143,16094.648286,16076.947713
2024-04-25,16089.820857,16084.993428,16093.039143,16081.775142
2024-04-26,16089.820857,16089.820857,16094.648286,16084.993428
2024-04-27,16084.993428,16089.820857,16093.039143,16083.384285
2024-04-28,16088.211714,16084.993428,16089.820857,16083.384285


In [51]:
# Langkah 2 : Proses Perhitungan Prediksi menggunakan DMA

short_window = 18
long_window = 20
data['SMA'] = data['Terakhir_IDR'].rolling(window=short_window, min_periods=1).mean()
data['LMA'] = data['Terakhir_IDR'].rolling(window=long_window, min_periods=1).mean()

# Prediction with Double Moving Average (DMA)
data['DMA_Prediction'] = data['SMA']

# Extend DMA predictions for future dates dynamically
last_values = data['SMA'].dropna().values[-short_window:]
future_dates = [data.index[-1] + datetime.timedelta(days=i) for i in range(1, 20)]
future_sma = []

for i in range(19):
    new_sma = np.mean(np.append(last_values, [last_values[-1]])) # Simulate next SMA value
    future_sma.append(new_sma)
    last_values = np.append(last_values[1:], new_sma) # Shift the window

future_df_dma = pd.DataFrame({'DMA_Prediction': future_sma}, index=future_dates)

# Combine the original and future DMA predictions
dma_data = pd.concat([data, future_df_dma])

# Calculate differences and percentage differences for DMA
dma_data['Difference'] = dma_data['Terakhir_IDR'] - dma_data['DMA_Prediction']
dma_data['Percentage_Difference'] = dma_data['Difference'] / dma_data['Terakhir_IDR'] * 100

# Display the DMA predictions
display(dma_data.tail(10))

# Plot DMA predictions using plotly backend
import plotly.express as px
fig = px.line(dma_data, x=dma_data.index, y=['Terakhir_IDR', 'DMA_Prediction'], title='Prediksi DMA USDT dengan Harga Asli')
fig.show()


,Terakhir_IDR,Pembukaan_IDR,Tertinggi_IDR,Terendah_IDR,SMA,LMA,DMA_Prediction,Difference,Percentage_Difference
2024-06-03,NaN,NaN,NaN,NaN,NaN,NaN,16092.132072,NaN,NaN
2024-06-04,NaN,NaN,NaN,NaN,NaN,NaN,16092.147040,NaN,NaN
2024-06-05,NaN,NaN,NaN,NaN,NaN,NaN,16092.133811,NaN,NaN
2024-06-06,NaN,NaN,NaN,NaN,NaN,NaN,16092.076056,NaN,NaN
2024-06-07,NaN,NaN,NaN,NaN,NaN,NaN,16091.984686,NaN,NaN
2024-06-08,NaN,NaN,NaN,NaN,NaN,NaN,16091.900854,NaN,NaN
2024-06-09,NaN,NaN,NaN,NaN,NaN,NaN,16091.855352,NaN,NaN
2024-06-10,NaN,NaN,NaN,NaN,NaN,NaN,16091.842409,NaN,NaN
2024-06-11,NaN,NaN,NaN,NaN,NaN,NaN,16091.868138,NaN,NaN
2024-06-12,NaN,NaN,NaN,NaN,NaN,NaN,16091.925488,NaN,NaN


In [44]:
# Langkah 3 : Proses Implementasi GRU, dimana akan menyiapkan Model GRU (dimana akan membagi Data latih dan Data Uji)

# Prepare data for GRU
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[['Terakhir_IDR']].fillna(method='ffill'))

# Create a function to prepare the dataset for GRU
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 20
X, Y = create_dataset(scaled_data, look_back)

# Reshape input to be [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Split into train and test sets
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size
X_train, X_test = X[0:train_size], X[train_size:len(X)]
Y_train, Y_test = Y[0:train_size], Y[train_size:len(Y)]

# Print shapes of training and testing data
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Y_train shape: {Y_train.shape}")
print(f"Y_test shape: {Y_test.shape}")


X_train shape: (8, 20, 1)
X_test shape: (2, 20, 1)
Y_train shape: (8,)
Y_test shape: (2,)


C:\Users\USER\AppData\Local\Temp\ipykernel_6572\684386128.py:5: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [45]:
#  Langkah 4 : Proses Implementasi GRU dimana akan melatih model GRU untuk memprediksi Pergerakan harga USDT

# Adjust the GRU model
model_path = 'GRU_model.h5'
if os.path.exists(model_path):
    model = load_model(model_path)
else:
    model = Sequential()
    model.add(GRU(50, return_sequences=True, input_shape=(look_back, 1), recurrent_activation='sigmoid'))
    model.add(GRU(50, recurrent_activation='sigmoid'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X_train, Y_train, epochs=20, batch_size=1, verbose=2)
    model.save(model_path)

# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Invert predictions
train_predict = scaler.inverse_transform(train_predict)
Y_train = scaler.inverse_transform([Y_train])
test_predict = scaler.inverse_transform(test_predict)
Y_test = scaler.inverse_transform([Y_test])

# Print some predictions
print(f"Train predictions: {train_predict[:5]}")
print(f"Test predictions: {test_predict[:5]}")


c:\Users\USER\anaconda3\envs\py_310\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning:

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.



Train predictions: [[16092.332 ]
 [16091.767 ]
 [16092.325 ]
 [16092.5625]
 [16093.161 ]]
Test predictions: [[16092.572]
 [16091.629]]


In [52]:
# Langkah Ke Enam : Menggunakan Model GRU untuk memprediksi
import datetime
import plotly.express as px

# Load the data and prepare future predictions
# Assuming scaled_data, look_back, model, and scaler are defined earlier

# Jumlah Hari
future_days = 18
future_predictions = []

last_sequence = scaled_data[-look_back:]
for _ in range(future_days):
    prediction = model.predict(np.reshape(last_sequence, (1, look_back, 1)))
    future_predictions.append(prediction[0, 0])
    last_sequence = np.append(last_sequence[1:], prediction, axis=0)

future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Prepare future dates for predictions
future_dates = [data.index[-1] + datetime.timedelta(days=i) for i in range(1, future_days + 1)]

# Append future predictions to the data
future_df_gru = pd.DataFrame(future_predictions, index=future_dates, columns=['GRU_Prediction'])

# Combine the original and future data
gru_data = data.copy()
gru_data['GRU_Prediction'] = np.nan
gru_data = pd.concat([gru_data, future_df_gru])

# Calculate differences and percentage differences for GRU
gru_data['Difference'] = gru_data['Terakhir_IDR'] - gru_data['GRU_Prediction']
gru_data['Percentage_Difference'] = gru_data['Difference'] / gru_data['Terakhir_IDR'] * 100

# Display the GRU predictions
print(gru_data.tail(10))

# Plot GRU predictions using Plotly
fig = px.line(gru_data, x=gru_data.index, y=['Terakhir_IDR', 'GRU_Prediction'], title='Prediksi GRU USDT dengan Harga Asli')
fig.show()


C:\Users\USER\AppData\Local\Temp\ipykernel_6572\1844523689.py:29: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



            Terakhir_IDR  Pembukaan_IDR  Tertinggi_IDR  Terendah_IDR  SMA  \
2024-06-02           NaN            NaN            NaN           NaN  NaN   
2024-06-03           NaN            NaN            NaN           NaN  NaN   
2024-06-04           NaN            NaN            NaN           NaN  NaN   
2024-06-05           NaN            NaN            NaN           NaN  NaN   
2024-06-06           NaN            NaN            NaN           NaN  NaN   
2024-06-07           NaN            NaN            NaN           NaN  NaN   
2024-06-08           NaN            NaN            NaN           NaN  NaN   
2024-06-09           NaN            NaN            NaN           NaN  NaN   
2024-06-10           NaN            NaN            NaN           NaN  NaN   
2024-06-11           NaN            NaN            NaN           NaN  NaN   

            LMA  DMA_Prediction  GRU_Prediction  Difference  \
2024-06-02  NaN             NaN    16091.769531         NaN   
2024-06-03  NaN           

In [53]:
# Langkah 7 : Membandingkan antara GRU, DMA dan harga Asli

# Combine DMA and GRU predictions
combined_data = dma_data.copy()
combined_data['GRU_Prediction'] = gru_data['GRU_Prediction']

# Display the combined predictions
display(combined_data.tail(10))

# Plot combined predictions using Plotly
fig = px.line(combined_data, x=combined_data.index, y=['Terakhir_IDR', 'DMA_Prediction', 'GRU_Prediction'], title='Perbandingan Prediksi USDT')
fig.show()

,Terakhir_IDR,Pembukaan_IDR,Tertinggi_IDR,Terendah_IDR,SMA,LMA,DMA_Prediction,Difference,Percentage_Difference,GRU_Prediction
2024-06-03,NaN,NaN,NaN,NaN,NaN,NaN,16092.132072,NaN,NaN,16091.925781
2024-06-04,NaN,NaN,NaN,NaN,NaN,NaN,16092.147040,NaN,NaN,16092.063477
2024-06-05,NaN,NaN,NaN,NaN,NaN,NaN,16092.133811,NaN,NaN,16092.183594
2024-06-06,NaN,NaN,NaN,NaN,NaN,NaN,16092.076056,NaN,NaN,16092.286133
2024-06-07,NaN,NaN,NaN,NaN,NaN,NaN,16091.984686,NaN,NaN,16092.376953
2024-06-08,NaN,NaN,NaN,NaN,NaN,NaN,16091.900854,NaN,NaN,16092.457031
2024-06-09,NaN,NaN,NaN,NaN,NaN,NaN,16091.855352,NaN,NaN,16092.525391
2024-06-10,NaN,NaN,NaN,NaN,NaN,NaN,16091.842409,NaN,NaN,16092.585938
2024-06-11,NaN,NaN,NaN,NaN,NaN,NaN,16091.868138,NaN,NaN,16092.639648
2024-06-12,NaN,NaN,NaN,NaN,NaN,NaN,16091.925488,NaN,NaN,NaN


In [54]:
# Langkah Ke Delapan : SImpan dalam bentuk Excel
# Save combined predictions to Excel
combined_data.to_excel('Tether_Combined_Predictions.xlsx')
print("Combined predictions saved to 'Tether_Combined_Predictions.xlsx'")


Combined predictions saved to 'Tether_Combined_Predictions.xlsx'
